### **Solution B:** Deep learning-based approaches that do not employ transformer architectures   
Our final model uses a BiMPM-inspired architecture with frozen RoBERTa embeddings. The model captures matching perspectives between the encoded premise and hypothesis via a custom multi-perspective matching layer. Pre-trained RoBERTa embeddings are computed once and used as static inputs. The model was optimized using Optuna, and trained on the full dataset.

---


#### **Expected Input**
- CSV file named `test.csv`
- Must contain two columns: `premise` and `hypothesis`  
- No labels required

#### **Expected Output**
- `group_33_B.csv` — one prediction per line (0 or 1)

---

#### **Instructions to Run**
1. Place your test file in `Data/test.csv`
2. Ensure the trained model is available at:  
   `savedModel/best_bimpm_model.keras`  
3. Run all cells in order to generate predictions

> **Note**: - The model uses precomputed RoBERTa embeddings (frozen) and a BiMPM-style architecture with multi-perspective matching.

> **Note**: This notebook is also used to predict on the `test.csv`, the predictions will be saved in `../predictions/group_33_b_csv`
 
---



Setup & Install Packages

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

!pip install transformers tensorflow --quiet
import re, string
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModel
from tensorflow.keras.layers import Layer
from keras.saving import register_keras_serializable

# test_path = "/content/drive/MyDrive/dataset/training_data/NLI/test.csv"
# model_path = "/content/drive/MyDrive/dataset/training_data/NLI/best_bimpm_model.keras"
# test_predictions = "/content/drive/MyDrive/dataset/training_data/NLI/Group_33_B.csv"

model_path = "savedModels/best_bimpm_model.keras"
test_path = "../Data/test.csv"
test_predictions = "../predictions/Group_33_B.csv"

MODEL_NAME = "roberta-base"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load and Preprocess Data

In [7]:
def clean_text(text):
    text = ''.join(ch for ch in text if ch not in string.punctuation)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

test_df = pd.read_csv(test_path)
test_df['premise'] = test_df['premise'].fillna("").apply(clean_text)
test_df['hypothesis'] = test_df['hypothesis'].fillna("").apply(clean_text)

Extract RoBERTa Embeddings

In [8]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
transformer = TFAutoModel.from_pretrained(MODEL_NAME)
transformer.trainable = False

def encode(df, max_len=50):
    return tokenizer(
        df['premise'].tolist(),
        df['hypothesis'].tolist(),
        padding="max_length", truncation=True, max_length=max_len,
        return_tensors="tf"
    )

def compute_embeddings(input_ids, attention_mask, batch_size=256):
    embeddings = []
    dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask)).batch(batch_size)
    for batch_ids, batch_mask in dataset:
        output = transformer(batch_ids, attention_mask=batch_mask).last_hidden_state
        embeddings.append(output.numpy())
    return np.vstack(embeddings)

test_encodings = encode(test_df)

test_embeddings = compute_embeddings(test_encodings['input_ids'], test_encodings['attention_mask'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

### Custom BiMPM Matching Layer

> Note:
This function is used inside a Lambda layer in the model architecture to split the inputs into premise and hypothesis embeddings.
Because the model was saved with this custom function, we need to **redefine it exactly as it was during training** so that Keras can correctly rebuild the model when loading it.


In [9]:
class BiMPMMatching(Layer):
    def __init__(self, hidden_size, num_perspectives, **kwargs):
        super().__init__(**kwargs)
        self.hidden_size = hidden_size
        self.num_perspectives = num_perspectives
        self.W = self.add_weight(
            shape=(num_perspectives, hidden_size * 2),
            initializer="random_normal",
            trainable=True,
            name="W_bimpm"
        )

    def call(self, inputs):
        premise_encoded, hypothesis_encoded = inputs

        def cosine_similarity(tensor_a, tensor_b):
            a_expanded = tf.expand_dims(tensor_a, axis=2) * tf.reshape(self.W, (1, 1, self.num_perspectives, self.hidden_size * 2))
            b_expanded = tf.expand_dims(tensor_b, axis=2) * tf.reshape(self.W, (1, 1, self.num_perspectives, self.hidden_size * 2))
            return -tf.keras.losses.cosine_similarity(a_expanded, b_expanded, axis=-1)

        def full_match(sequence, last_step_other_sequence):
            last_step_other_sequence_expanded = tf.repeat(tf.expand_dims(last_step_other_sequence, 1), tf.shape(sequence)[1], axis=1)
            return cosine_similarity(sequence, last_step_other_sequence_expanded)

        def maxpool_match(sequence_a, sequence_b):
            pooled_similarities = []
            for i in range(sequence_a.shape[1]):
                sequence_a_i = tf.repeat(tf.expand_dims(sequence_a[:, i, :], 1), tf.shape(sequence_b)[1], axis=1)
                similarity_scores = cosine_similarity(sequence_a_i, sequence_b)
                pooled_similarities.append(tf.reduce_max(similarity_scores, axis=1))
            return tf.stack(pooled_similarities, axis=1)

        full_match_premise = full_match(premise_encoded, hypothesis_encoded[:, -1, :])
        full_match_hypothesis = full_match(hypothesis_encoded, premise_encoded[:, -1, :])
        maxpool_premise = maxpool_match(premise_encoded, hypothesis_encoded)
        maxpool_hypothesis = maxpool_match(hypothesis_encoded, premise_encoded)

        return tf.concat([full_match_premise, full_match_hypothesis, maxpool_premise, maxpool_hypothesis], axis=-1)

@register_keras_serializable()
def split_premise_and_hypothesis(x):
    return tf.split(x, num_or_size_splits=2, axis=1)

Load Model and Predict

In [10]:
print("Loading model from:", model_path)

model = tf.keras.models.load_model(model_path, custom_objects={
    'BiMPMMatching': BiMPMMatching,
    'split_premise_and_hypothesis': split_premise_and_hypothesis
})

print("Predicting...")
predictions = model.predict(test_embeddings).argmax(axis=1)


Loading model from: /content/drive/MyDrive/dataset/training_data/NLI/best_bimpm_model.keras
Predicting...
104/104 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step


Save Test Predictions

In [14]:
pd.DataFrame({'prediction': predictions}).to_csv(test_predictions, index=False)

print("Saved predictions to:", test_predictions)

Saved predictions to: /content/drive/MyDrive/dataset/test_data/NLI/group_33_B.csv
